# Imputation and Final Modeling

Date: 27th Nov 2019

In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
### Read and filter unwanted data
df_p3app=pd.read_csv('C:/Users/Tejas/Downloads/David JnJ/DataBank/p3app_unonehot.csv')

df_p3app=df_p3app[~df_p3app['trialTiming.totalTrialDuration'].isnull()].reset_index(drop=True).drop([
    'Unnamed: 0','Unnamed: 0_x','Unnamed: 0_y','trialId','drugId','trialSponsors','trialTherapeuticAreas', 'trialPhase'],1)

df_p3app['globalStatus']=((df_p3app['globalStatus']=='Launched')|(df_p3app['globalStatus']=='Registered'))*1


In [3]:
#### Take a glipse of the data
df_p3app.head(10)

,deliveryRoutes,Origin,deliveryMediums,targets1,pharmaTarget_families,Active,Approved,globalStatus,trialTiming.totalTrialDuration,trialStudyKeywords,...,iphase2pos,iphase2neg,iphase3pos,iphase3neg,iphase1com,iphase1ter,iphase2com,iphase2ter,iphase3com,iphase3ter
0,['Oral'],others,"['Capsule, hard', 'Solution', 'Tablet']",['Receptor'],[],1,1,1,47.84,"['double blind/blinded', 'multiple arm', 'plac...",...,15,2,17,4,9,0,35,0,49,3
1,['Oral'],others,"['Capsule, hard', 'Solution', 'Tablet']",['Receptor'],[],1,1,1,3.81,"['double blind/blinded', 'multiple arm', 'plac...",...,0,0,0,0,0,0,0,0,0,0
2,['Oral'],others,"['Capsule, hard', 'Solution', 'Tablet']",['Receptor'],[],1,1,1,5.95,"['double blind/blinded', 'multiple arm', 'plac...",...,0,0,0,0,0,0,0,0,0,0
3,['Oral'],others,"['Capsule, hard', 'Solution', 'Tablet']",['Receptor'],[],1,1,1,18.43,"['active comparator', 'double blind/blinded', ...",...,14,2,22,4,3,0,30,1,43,1
4,['Oral'],others,"['Capsule, hard', 'Solution', 'Tablet']",['Receptor'],[],1,1,1,7.69,"['active comparator', 'double blind/blinded', ...",...,4,0,18,4,1,0,5,1,27,0
5,['Oral'],others,"['Capsule, hard', 'Solution', 'Tablet']",['Receptor'],[],1,1,1,4.76,"['open label', 'safety', 'single arm']",...,1,0,3,0,0,0,2,0,4,0
6,['Oral'],others,"['Capsule, hard', 'Solution', 'Tablet']",['Receptor'],[],1,1,1,4.99,"['active comparator', 'double blind/blinded', ...",...,6,0,17,1,4,0,23,0,29,0
7,['Oral'],others,"['Capsule, hard', 'Solution', 'Tablet']",['Receptor'],[],1,1,1,8.44,"['active comparator', 'double blind/blinded', ...",...,2,0,13,2,0,0,3,1,25,3
8,['Oral'],others,"['Capsule, hard', 'Solution', 'Tablet']",['Receptor'],[],1,1,1,28.95,"['double blind/blinded', 'efficacy', 'multiple...",...,0,0,0,0,0,0,0,0,0,0
9,['Inhaled'],others,['Solution'],['Receptor'],[],1,1,1,9.13,"['double blind/blinded', 'efficacy', 'placebo ...",...,77,15,162,12,21,1,154,8,323,9


In [4]:
### Check all the missing Values
df_p3app.isnull().sum().reset_index()
### Left side is the column number and right side is the number of missing values in that column

,index,0
0,deliveryRoutes,374
1,Origin,0
2,deliveryMediums,5723
3,targets1,4864
4,pharmaTarget_families,0
5,Active,0
6,Approved,0
7,globalStatus,0
8,trialTiming.totalTrialDuration,0
9,trialStudyKeywords,49


In [5]:
df_p3app.info()
## The ones that are given as object in the right side are categorical Variables, the rest are numerical variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27357 entries, 0 to 27356
Data columns (total 53 columns):
deliveryRoutes                    26983 non-null object
Origin                            27357 non-null object
deliveryMediums                   21634 non-null object
targets1                          22493 non-null object
pharmaTarget_families             27357 non-null object
Active                            27357 non-null int64
Approved                          27357 non-null int64
globalStatus                      27357 non-null int32
trialTiming.totalTrialDuration    27357 non-null float64
trialStudyKeywords                27308 non-null object
sponsorType                       27357 non-null object
TherapeuticArea                   27357 non-null object
TrialStatus                       27357 non-null object
trialOutcomes                     27357 non-null object
trialActualAccrual                26523 non-null float64
trialTargetAccrual                26349 non-null flo

###########################################################################################################################

## From the above information, we can observe that: </br>
##### The numbers mentioned are the column numbers.
All_categorical = 0, 1, 2, 3, 4, 9, 10, 11, 12, 13, 16, 18

categorical_list = 0, 2, 3, 4, 9, 10, 13, 16, 18
categorical_nonlist = 1, 11, 12

Note: None of the categorical non_list values have missingness

In [8]:
### Lets one hot encode the sampeles. 
### 1. The ones in the form of lists need to be encoded in a different way
### 2. The ones with categorical variables but not in form of lists in a different way
### 3. The numeric ones, just copy paste as it is

from sklearn.preprocessing import MultiLabelBinarizer

categorical_list = [0, 2, 3, 4, 9, 10, 13, 16, 18]
categorical_nonlist = [1, 11, 12]

df_p3app_new=pd.DataFrame()
for i in range(df_p3app.shape[1]):
    
    if i in categorical_list:
        temp=df_p3app[df_p3app.columns[i]].apply(lambda x:ast.literal_eval(x) if x==x and x!='[]' else [df_p3app.columns[i]+'_others'] 
                                                 if x=='[]' else ["Missing"])
        mlb=MultiLabelBinarizer()
        temp=pd.DataFrame(mlb.fit_transform(temp),columns=mlb.classes_)
        if "Missing" in temp.columns:
            temp.loc[temp['Missing']==1,:]=np.nan
            temp=temp.drop('Missing',1)
        df_p3app_new=pd.concat([df_p3app_new,temp],1)
    
    
    if i in categorical_nonlist:
        
        temp = pd.get_dummies(df_p3app[df_p3app.columns[i]])
        df_p3app_new=pd.concat([df_p3app_new,temp],1)
    
    else:
        df_p3app_new[df_p3app.columns[i]]=df_p3app[df_p3app.columns[i]]


In [10]:
### Lets take a glance at encoded data
df_p3app_new.tail(20)   ### NaN might not be seen below, but they exist haha 

,Inhaled,Injectable,Oral,Topical,deliveryRoutes_others,deliveryRoutes,"Biological, protein, antibody","Biological, protein, recombinant","Chemical, synthetic",others,...,iphase2pos,iphase2neg,iphase3pos,iphase3neg,iphase1com,iphase1ter,iphase2com,iphase2ter,iphase3com,iphase3ter
27337,0.0,0.0,0.0,0.0,1.0,[],0,0,0,1,...,204,134,124,101,126,39,748,253,403,132
27338,0.0,0.0,0.0,0.0,1.0,[],0,0,0,1,...,0,0,0,0,2,0,9,0,8,0
27339,0.0,0.0,0.0,0.0,1.0,[],0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
27340,0.0,0.0,1.0,0.0,0.0,['Oral'],0,0,0,1,...,0,0,2,0,0,0,3,0,6,0
27341,0.0,0.0,0.0,0.0,1.0,[],0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
27342,0.0,0.0,0.0,0.0,1.0,[],0,0,0,1,...,27,7,32,15,8,4,62,4,71,3
27343,0.0,0.0,0.0,0.0,1.0,[],0,0,0,1,...,0,0,2,0,0,0,0,0,3,0
27344,0.0,0.0,0.0,0.0,1.0,[],0,0,0,1,...,66,47,139,59,43,2,202,29,305,43
27345,0.0,0.0,0.0,0.0,1.0,[],0,0,0,1,...,10,8,16,14,3,0,40,8,60,14
27346,0.0,0.0,1.0,0.0,0.0,['Oral'],0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
### In the above loop, the categorical var with list columns are also in the new dataset, which is not needed anymore,
categorical_listnames =  ['deliveryRoutes', 'deliveryMediums', 'targets1', 'pharmaTarget_families', 'trialStudyKeywords',
                          'sponsorType', 'trialOutcomes', 'trialCountries', 'trialTags']

### remove categorical list name columns from new df. 
df_p3app_new.drop(categorical_listnames, axis = 1, inplace = True)
df_p3app_new.reset_index(drop = True, inplace = True)


## Data Cleaning is done, Let's jump into Imputation 

Here, there are 2 packages available in python which could do KNN imputation. Yuxi was using "missingpy KNN" package, and had adjusted the internal code to work for our necessity. The other package is "fancyimpute KNN" package. <br> 

How to install them has been detailed below. Kindly let me know if there is any confusion in them. 

<b>missingpy KNN:</b> 
1. Yuxi has uploaded missingpy files in the gdrive. 
2. First, pip install missingpy in command prompt.
3. Open any python compiler, and type help('modules'), here you could find the path address to where your installed packages are located.
4. Go to that folder and replace the files Yuxi has uploaded in gdrive
5. Come back to your python compiler, restart kernel, and then you are good to go.

<b> fancyimpute KNN: </b> (installing this package is quite the circus)

1. So first you need to have visual studio builder tools installed in your laptop.
2. conda install ecos
3. conda install CVXcanon
4. pip install fancy impute

Note: This isn't as easy as it seems to be. It took 5-6 hours to figure this thing haha. I did above 4 installations like a million times, and then I have no clue what changed, suddenly it started working somehow. So, good luck.

In [12]:
#### Missingpy package
#### so missingpy has that error, but fancy impute works.
from missingpy import KNNImputer

imputer1 = KNNImputer(n_neighbors = 3, metric='gower', n_jobs=1)
imp = imputer1.fit(df_p3app_new.iloc[:1000])

from time import time
t = time()
df_p3app_imp = imp.transform(df_p3app_new.iloc[:1000])
print(time()-t)

ValueError: Cannot index with multidimensional key

## The above error I think is that some column names have 2 dimensions, meaning like this may be:

                  trialDuration
trialID  DrugID                  global_status
12          3         56             1


I tried checking about this, but i couldn't find any such columns, this error never got solved. Hence I shifted to fancyimpute for the time being.

In [13]:
### FancyImpute - The below code takes 15 mins, kindly be patient

col = list(df_p3app_new.columns)         # Here, in fancy impute, after imputation u would get a matrix with no col names.
                                         # Hence, We are trieving a memory of col names to put them back together.


### This works, but it isnt gower distance
from fancyimpute import KNN

imputer = KNN(k = 5)
df_p3app_imp = imputer.fit_transform(df_p3app_new)


C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])

Imputing row 1/27357 with 6 missing, elapsed time: 595.546
Imputing row 101/27357 with 6 missing, elapsed time: 595.890
Imputing row 201/27357 with 0 missing, elapsed time: 596.031
Imputing row 301/27357 with 6 missing, elapsed time: 596.200
Imputing row 401/27357 with 6 missing, elapsed time: 596.325
Imputing row 501/27357 with 6 missing, elapsed time: 596.468
Imputing row 601/27357 with 6 missing, elapsed time: 596.578
Imputing row 701/27357 with 0 missing, elapsed time: 596.593
Imputing row 801/27357 with 0 missing, elapsed time: 596.671
Imputing row 901/27357 with 6 missing, elapsed time: 596.748
Imputing row 1001/27357 with 6 missing, elapsed time: 596.880
Imputing row 1101/27357 with 7 missing, elapsed time: 596.968
Imputing row 1201/27357 with 13 missing, elapsed time: 597.078
Imputing row 1301/27357 with 1 missing, elapsed time: 597.252
Imputing row 1401/27357 with 0 missing, elapsed time: 597.373
Imputing row 1501/27357 with 7 missing, elapsed time: 597.504
Imputing row 1601/2

Imputing row 13401/27357 with 7 missing, elapsed time: 610.531
Imputing row 13501/27357 with 7 missing, elapsed time: 610.652
Imputing row 13601/27357 with 12 missing, elapsed time: 610.754
Imputing row 13701/27357 with 0 missing, elapsed time: 610.884
Imputing row 13801/27357 with 6 missing, elapsed time: 610.994
Imputing row 13901/27357 with 7 missing, elapsed time: 611.065
Imputing row 14001/27357 with 7 missing, elapsed time: 611.147
Imputing row 14101/27357 with 0 missing, elapsed time: 611.184
Imputing row 14201/27357 with 7 missing, elapsed time: 611.260
Imputing row 14301/27357 with 0 missing, elapsed time: 611.326
Imputing row 14401/27357 with 0 missing, elapsed time: 611.398
Imputing row 14501/27357 with 7 missing, elapsed time: 611.506
Imputing row 14601/27357 with 13 missing, elapsed time: 611.612
Imputing row 14701/27357 with 7 missing, elapsed time: 611.670
Imputing row 14801/27357 with 0 missing, elapsed time: 611.813
Imputing row 14901/27357 with 12 missing, elapsed tim

Imputing row 26601/27357 with 0 missing, elapsed time: 623.964
Imputing row 26701/27357 with 19 missing, elapsed time: 624.051
Imputing row 26801/27357 with 7 missing, elapsed time: 624.146
Imputing row 26901/27357 with 0 missing, elapsed time: 624.261
Imputing row 27001/27357 with 0 missing, elapsed time: 624.336
Imputing row 27101/27357 with 0 missing, elapsed time: 624.437
Imputing row 27201/27357 with 19 missing, elapsed time: 624.543
Imputing row 27301/27357 with 14 missing, elapsed time: 624.694


In [14]:
### Convert the imputed matrix into dataframe and add the column names back again
df_p3app_imp = pd.DataFrame(df_p3app_imp)
df_p3app_imp.columns = col


### Here, we see that in certain imputations, decimal values have been imputed in one-hot columns. Hence, as a temporary solution I thought of rounding the imputed values to 0 or 1, and then using the data for modelling purpose.

In our original data except for trialDuration, nothing else is in decimal format.
So now planing to round the decimals in whole data other than trialDuration.

In [15]:
trial_duration = df_p3app_imp['trialTiming.totalTrialDuration']

df_p3app_imp = df_p3app_imp.round()

df_p3app_imp['trialTiming.totalTrialDuration'] = trial_duration   ### adding original values back into array



### So first I executed the code with all the variables, then I got an accuracy of 98%. With that impractical accuracy when i was looking to see what went wrong, i saw that column in drug data which checks Active/Ceased column is almost the same as GlobalStatus, hence that high accuracy. So now we are removing those 2 columns to check for accuracies.

In [24]:
df_p3app_imp.drop(['Active', 'Approved'], axis = 1 ,inplace = True) ## removing those 2 cols with same info as global status

# Imputation Complete, Lets Model It :) :) 

In [25]:
target_variable = 'globalStatus'

X = np.array(df_p3app_imp.drop([target_variable], 1))  ## all features except global status
y = np.array(df_p3app_imp[target_variable])            ## globalStatus

### Initially I am building models just to run once, and then I have written a loop to check its robustness against 100 times.

In [26]:
from sklearn import model_selection
from sklearn import metrics

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.20)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(21885, 160) (21885,)
(5472, 160) (5472,)


In [27]:
### A function to generate AUC score.
def AUC_score(modl, y_test, y_pred):
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    AUC_score = metrics.auc(fpr, tpr)
    return AUC_score

### SVM Model. I am using LinearSVC since, using kernel = 'rbf' SVM takes very high computation time to generate results. In the same way, I am using a very basic RandomForest. I will do the hyperparameter optimization later makes the models more powerful for a better accuracy.

In [28]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

modl = svm.LinearSVC(class_weight = 'balanced').fit(X_train, y_train)
y_pred = modl.predict(X_test)
AUC_score(modl, y_test, y_pred)


C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.5485239049813753

In [29]:
modl2 = RandomForestClassifier(n_estimators = 100).fit(X_train, y_train)
y_pred2 = modl2.predict(X_test)
AUC_score(modl2, y_test, y_pred2)

0.5454895480987345

In [30]:
#### Now let us build a loop with 100 monte carlo runs and then average the accuracies. takes 20 mins

AUC_svm = []
AUC_rf = []

for i in range(0, 100):
    
    ### create random datasets each time
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.20)
    
    ###Build SVM and store the result
    modl = svm.LinearSVC(class_weight = 'balanced').fit(X_train, y_train)
    y_pred = modl.predict(X_test)
    AUC_svm.append(AUC_score(modl, y_test, y_pred))
    
    ### Build Random forest and store the results
    modl2 = RandomForestClassifier(n_estimators = 100).fit(X_train, y_train)
    y_pred2 = modl2.predict(X_test)
    AUC_rf.append(AUC_score(modl2, y_test, y_pred2))
    
    
    

C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:9

C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:9

C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Tejas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:9

KeyboardInterrupt: 

In [31]:
print(AUC_svm)

[0.6375000267932357, 0.5131636562671045, 0.5479273824723654, 0.5899754883186519, 0.5522558192150548, 0.5303469934890847, 0.47750910659509205, 0.5755133691583536, 0.4784251244733818, 0.5611537927064102, 0.5868046979202688, 0.5807737754907567, 0.599374460108942, 0.6015953049677516, 0.5051936359879567, 0.5645949427347315, 0.5567733990147783, 0.5531201241243751, 0.5300125885342012, 0.5549003962139556, 0.5479590762545186, 0.5154581447963801, 0.5645979964496303, 0.5191219596548187, 0.5467427309532572, 0.5023477866219801, 0.5432429783540086, 0.5126267230755369, 0.5428599287394521, 0.628232731000897, 0.5680812634762398, 0.6532570951341372, 0.4957018345003326, 0.542200094469851, 0.5183736353307317, 0.5401853367938826, 0.5755563612163712, 0.6193383905481312, 0.5253099356514552, 0.49278305252181553, 0.5890451072336338, 0.5300127713920817, 0.5176390183777634, 0.5214501288185498, 0.6208939166238647, 0.507248863992056, 0.5547325070537659, 0.5294260619362048, 0.5521163047478836, 0.5592173865320235, 0

In [32]:
print(AUC_rf)

[0.5521760751410932, 0.5431855500821018, 0.5504134374863802, 0.5308295672156261, 0.5545555570342965, 0.5434890846418997, 0.5418903374233128, 0.5365161556830247, 0.539680582152432, 0.5420397170441837, 0.5576248579755966, 0.5459005145797599, 0.540835793002298, 0.5398376115581017, 0.5481861384969596, 0.563269487459513, 0.5372331691297209, 0.5341619661275622, 0.5482068676125651, 0.5419546555139776, 0.5522346518914244, 0.537183257918552, 0.5480303476329381, 0.5505683084643019, 0.5526757453073243, 0.5506737885770143, 0.5518631047389693, 0.54114486924527, 0.5466992227968749, 0.5446174548250673, 0.5506683281528567, 0.5486256611624738, 0.5478616288378491, 0.5536318621320121, 0.5503260386306378, 0.5594691636745337, 0.5464692072415822, 0.5333284750292071, 0.5628032941732097, 0.5472531694199001, 0.545280978279967, 0.5446237733177165, 0.5248731886920658, 0.5539197644460803, 0.5299941451124371, 0.5458547852648167, 0.5373061866652566, 0.5413698274367016, 0.5523739418476261, 0.5376625871247462, 0.5546

In [33]:
### Mean avg of SVM scores
np.mean(AUC_svm)

0.5472033551236954

In [34]:
### Mean Avg of RandomForest Scores
np.mean(AUC_rf)

0.546425157266451